In [9]:
import pandas as pd
from tqdm import tqdm
import numpy as np

In [10]:
file23 = './data/230619.csv'


In [11]:
df23 = pd.read_csv(file23, encoding='utf-8')
df23.head()

,the_datetime,symbol,track,sharpe_ratio_modified_3m,sharpe_ratio_modified_6m,sharpe_ratio_modified_1y,sharpe_ratio_modified_2y,annual_returns_3m,annual_returns_6m,annual_returns_1y,...,sec_ind_ex_avg_cximp_6m,sec_ind_ex_avg_cximp_1y,sec_ind_ex_avg_cximp_2y,comp_winning_yield_3m_1y,comp_winning_yield_3m_2y,ind_ex_avg_cximp_3m,ind_ex_avg_cximp_6m,ind_ex_avg_cximp_1y,ind_ex_avg_cximp_2y,daily_return
0,2012-12-31,1,价值,0.483971,-0.266978,0.314474,0.339701,0.233459,0.353288,0.351001,...,0.138174,-0.042008,NaN,1.565427,0.278980,-0.195429,-0.085693,-0.257223,NaN,0.012645
1,2012-12-31,11,价值,-0.375688,-0.538772,-0.239646,0.514503,-0.133633,-0.186934,0.096457,...,0.334712,0.581397,NaN,0.388653,0.364187,-0.464664,-0.055407,0.405757,NaN,0.009002
2,2012-12-31,21,均衡,0.288329,-0.551492,-0.791903,-1.201647,0.431506,-0.457290,-0.651966,...,-0.755362,-0.785504,NaN,-0.557381,-1.108675,-0.880620,-0.953606,-0.746467,NaN,0.011981
3,2012-12-31,31,均衡,-0.170698,-0.555539,-0.803354,-1.198081,-0.059994,-0.493962,-0.883196,...,0.036506,-0.224371,NaN,-2.172562,-1.863365,-0.530784,-0.569187,-0.615701,NaN,0.012488
4,2012-12-31,61,价值,0.864647,-0.115932,0.860236,-0.527318,1.320411,0.485371,0.981550,...,0.490441,0.536900,NaN,0.388653,-0.646123,0.033425,0.086260,0.079238,NaN,0.014706


In [12]:
df = df23.copy()

In [13]:
# 将分类变量转换为虚拟变量
dummy_df = pd.get_dummies(df['track'])
# 将虚拟变量拼接到原始 DataFrame
df = pd.concat([df, dummy_df], axis=1)
df.drop('track', axis=1, inplace=True)
# 将缺失值填补为0
df.fillna(0, inplace=True)

In [16]:
df = df.replace({True: 1, False: 0})

In [17]:
df.head()

,the_datetime,symbol,sharpe_ratio_modified_3m,sharpe_ratio_modified_6m,sharpe_ratio_modified_1y,sharpe_ratio_modified_2y,annual_returns_3m,annual_returns_6m,annual_returns_1y,annual_returns_2y,...,价值,其他,制造,医药,周期,均衡,成长,消费,科技,金融
0,2012-12-31,1,0.483971,-0.266978,0.314474,0.339701,0.233459,0.353288,0.351001,0.108795,...,1,0,0,0,0,0,0,0,0,0
1,2012-12-31,11,-0.375688,-0.538772,-0.239646,0.514503,-0.133633,-0.186934,0.096457,0.841372,...,1,0,0,0,0,0,0,0,0,0
2,2012-12-31,21,0.288329,-0.551492,-0.791903,-1.201647,0.431506,-0.457290,-0.651966,-0.946475,...,0,0,0,0,0,1,0,0,0,0
3,2012-12-31,31,-0.170698,-0.555539,-0.803354,-1.198081,-0.059994,-0.493962,-0.883196,-1.089881,...,0,0,0,0,0,1,0,0,0,0
4,2012-12-31,61,0.864647,-0.115932,0.860236,-0.527318,1.320411,0.485371,0.981550,-0.258823,...,1,0,0,0,0,0,0,0,0,0


In [18]:

grouped_dfs = df.groupby('symbol')
dfs = []
weeks_data = []
weeks_return = []
T=50
# Iterate over each group 
for group_name, group_df in tqdm(grouped_dfs):
    group_df['the_datetime'] = pd.to_datetime(group_df['the_datetime'])
    group_df.set_index('the_datetime', inplace=True)

    weekly_returns = group_df['daily_return'].resample('W-MON', label='left', closed='left').sum()
    group_df['weekly_return'] = weekly_returns
    
    

    ys = group_df['weekly_return'].values
    xs = group_df.drop("weekly_return", axis=1).values

    weeks_return_idx = []
    for i, week_return in enumerate(ys):
        if np.isnan(week_return) == False:# and i >= T:
            weeks_return.append(week_return)
            weeks_return_idx.append(i)
            
    # print("y len:", len(weeks_return_idx))
    for idx in weeks_return_idx:
        if idx < T:
            week_data = xs[0:idx]
            x = np.zeros((T-week_data.shape[0],119))
            week_data = np.concatenate((x, week_data), axis=0)
            weeks_data.append(week_data)
        else:
            week_data = xs[idx-T:idx]
            weeks_data.append(week_data)
    # print("x len:", len(weeks_data))

    dfs.append(group_df)

# Combine the groups back into a single DataFrame
df = pd.concat(dfs, axis=0)

# Reset the index
df.reset_index(drop=True, inplace=True)

  0%|          | 0/3791 [00:00<?, ?it/s]

100%|██████████| 3791/3791 [00:54<00:00, 69.57it/s] 


In [19]:
np.save('X.npy', np.array(weeks_data))
np.save('y.npy', np.array(weeks_return))

In [65]:
columns_with_nulls = df.columns[df.isnull().any()].tolist()
null_percentages = df.isnull().mean()
null_columns_count = (df.isnull().sum() > 0).sum()

print(null_columns_count)
print(null_percentages)

1
symbol                      0.0000
sharpe_ratio_modified_3m    0.0000
sharpe_ratio_modified_6m    0.0000
sharpe_ratio_modified_1y    0.0000
sharpe_ratio_modified_2y    0.0000
                             ...  
成长                          0.0000
消费                          0.0000
科技                          0.0000
金融                          0.0000
weekly_return               0.8056
Length: 120, dtype: float64


In [66]:
df.to_csv('230619_preprocessed.csv', index=False)